**Importar librerías necesarias**

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.0.0-bin-hadoop2.7"

In [2]:
import findspark
findspark.init()

In [24]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()
sc=spark.sparkContext

import numpy as np
#np.set_printoptions(threshold=sys.maxsize)

**Segmento donde se cambian las palabras del comentario por el índice: (También ordena y agrupa)**

In [4]:
#Arreglo que contiene las palabras del diccionario.
diccionario = (
    np
    .loadtxt('../resultados/300_com/indice_300/part-00000', 
    delimiter= ',', 
    dtype=str)
    )

In [5]:
#RDD con los comentarios de la BD.
#comentarios = np.loadtxt('/home/jsarabia/Documents/IA/analisis_sentimientos/Base/BD_100.csv', delimiter= ',', dtype=str)
comentarios = (
    sc.textFile('../Base/BD_300.csv')
    .map(lambda s: s.split(','))
    .map(lambda col: col[0])
    ).zipWithIndex().filter(lambda col: col[1] > 0).map(lambda col: col[0])
#comentarios.cache()

In [6]:
comentarios.take(8)

['EXCELENTE ESPACIO DE DISCUSION GRACIAS POR LA INVITACION',
 'EL PRESIDENTE SENALA QUE ES MUY IMPORTANTE QUE EXISTAN CAMPANAS DE ORIENTACION A LOS JOVENES PARA QUE NO SE CONSUMA DROGA PARA QUE NO AUMENTE EL CONSUMO',
 'POS CLARO',
 'MEXICO ARDE EL GOBIERNO ES RESPONSABLE TE INVITO A LEER MI COLUMNA',
 'EL PRIMER RECURSO DEBE SER SIEMPRE LA CONTENCION DE QUIEN ESTA ATRAVESANDO LA CURVA DE LA VIOLENCIA DESDE LO TECNICO DEPENDE EL CASO PUEDE SER GESTIONAR LA BAJA DEL CONTENIDO LESIVO DOCUMENTAR LA AGRESION TEJER REDES DE APOYO FRENTE A ATAQUES TROLLS',
 'ARMONIA',
 'LES QUERIA CONTAR QUE EN EL POR PRIMERA VEZ DESDE SU EXISTENCIA EL TALLER DE MUJERES CIENCIA Y TECNOLOGIA TUVO QUE DIVIDIRSE EN COMISIONES DEBIDO A LA CANTIDAD DE ASISTENTES ESTA IMAGEN ES TODO',
 'NOS RE ENCANTO LEAN LEAN LEAN TODA LA CUENTA DE ESTA GENIAL ESPANOLA ES DIGNA DE SER SEGUIDA GRACIAS Y SALUDOS DESDE BUENOS AIRES ARGENTINA']

In [7]:
def agruparElementos(lista):
    agrupados = []
    cont = 1
    for i in range(len(lista)):
        if(i < len(lista) - 1):
            if(lista[i] == lista[i + 1]):
                cont += 1
            else:
                agrupados.append([lista[i], cont])
                cont = 1
        else:
            agrupados.append([lista[i], cont])
    arr_ag = np.array(agrupados)
    return arr_ag

In [10]:
"""def llenarArray(lista):
    arr = []
    agrupados = agruparElementos(lista)
    for i in range(1, len(diccionario) + 1):
        if(np.where(agrupados[:, 0] == [i])[0] != None):
            indice = np.where(agrupados[:, 0] == [i])[0]
            repeticiones = agrupados[indice, 1]
            arr.append(str(i) + ':' +str(repeticiones))
        else:
            arr.append(str(i) + ':0')  
    array_numpy = np.array(arr)
    return array_numpy"""

In [8]:
def llenarArray(lista):
    arr = []
    agrupados = agruparElementos(lista)
    for i in range(1, len(diccionario) + 1):
        if(np.where(agrupados[:, 0] == [i])[0] != None):
            indice = np.where(agrupados[:, 0] == [i])[0]
            repeticiones = agrupados[indice, 1]
            arr.append(str(i) + ':' +str(repeticiones))
    array_numpy = np.array(arr)
    return array_numpy

In [9]:
def sustituir2(comentario):
    #Las palabras por comentario serán referidas en el arreglo palabras.
    palabras = comentario.split(' ')
    lista = []
    for palabra in range(len(palabras)):
        indice = np.where(diccionario == palabras[palabra])[0] + 1
        lista.append(indice)
    lista.sort()
    #El método llenarArray() retorna un arreglo de Numpy.
    arreglo = llenarArray(lista) 
    return arreglo

In [25]:
cad2 = 'EL PRESIDENTE SENALA QUE ES MUY IMPORTANTE QUE EXISTAN CAMPANAS DE ORIENTACION A LOS JOVENES PARA QUE NO SE CONSUMA DROGA PARA QUE NO AUMENTE EL CONSUMO'
arr_prueba = sustituir2(cad2)
print(arr_prueba)

['1:[1]' '3:[2]' '4:[4]' '6:[1]' '8:[2]' '10:[2]' '11:[1]' '13:[1]'
 '14:[1]' '51:[1]' '61:[1]' '189:[1]' '305:[1]' '306:[1]' '580:[1]'
 '581:[1]' '582:[1]' '583:[1]' '1403:[1]' '1404:[1]' '1405:[1]']


In [22]:
rdd_agrupados = comentarios.map(lambda s: sustituir2(s))
#rdd_agrupados.take(1)

In [14]:
arr_final = np.array(rdd_agrupados.collect())

In [35]:
def crearIndice():    
    lista = []
    for i in range(100):
        lista.append(i)
    rdd_indice = sc.parallelize(lista)
    return rdd_indice


In [15]:
np.savetxt("../resultados/300_com/cluster.txt", arr_final, fmt='%s')

In [23]:
rdd_agrupados.coalesce(1).saveAsTextFile('../resultados/300_com/palabras_etiquetadas_300')

# Segmento de Prueba